**Query 1**
I first selected all ICD codes for major abdominal/leg/carotid cases and filtered out abd cases that likely required clamping of renal arteries. This would likely cause AKI so this patient population was excluded. Patients were classified as endo or open.

-- ------------------------------------------------------------------
-- SCRIPT 1: PREVIEW_STRICT_COHORT
-- ------------------------------------------------------------------

WITH vascular_scans AS (
    SELECT 
        subject_id,
        hadm_id,
        icd_code,
        icd_version,
        CASE
            -- === GROUP A: OPEN REVASCULARIZATION ===
            
            -- 1. OPEN AAA (Target: Abdominal Aorta '0')
            WHEN icd_version = 10 AND (icd_code LIKE '04B0%' OR icd_code LIKE '04R0%') THEN 'OPEN_AAA'
            WHEN icd_version = 9  AND (icd_code = '3844') THEN 'OPEN_AAA'
            
            -- 2. OPEN LEG BYPASS (Target: Aorta/Iliac/Leg; Exclude: Visceral 1-8)
            WHEN icd_version = 10 AND icd_code LIKE '041%' 
                 AND SUBSTRING(icd_code, 4, 1) NOT IN ('1','2','3','4','5','6','7','8') THEN 'OPEN_LE_BYPASS'
            WHEN icd_version = 9  AND (icd_code LIKE '3925' OR icd_code LIKE '3929') THEN 'OPEN_LE_BYPASS'
            
            -- 3. OPEN CAROTID (Target: Common K/L, Internal M/N)
            WHEN icd_version = 10 AND icd_code LIKE '03C%' 
                 AND SUBSTRING(icd_code, 4, 1) IN ('K','L','M','N') THEN 'OPEN_CEA'
            WHEN icd_version = 9  AND icd_code = '3812' THEN 'OPEN_CEA'
            
            -- === GROUP B: ENDOVASCULAR ===
            
            -- 4. EVAR (Target: Abdominal Aorta '0')
            WHEN icd_version = 10 AND icd_code LIKE '04U0%' THEN 'ENDO_EVAR'
            WHEN icd_version = 9  AND icd_code = '3971' THEN 'ENDO_EVAR'
            
            -- 5. PERIPHERAL ENDO (Target: Leg; Exclude: Visceral 1-8)
            WHEN icd_version = 10 AND (icd_code LIKE '047%' OR icd_code LIKE '04V%') 
                 AND SUBSTRING(icd_code, 4, 1) NOT IN ('1','2','3','4','5','6','7','8') THEN 'ENDO_PVI'
            WHEN icd_version = 9  AND (icd_code = '3990' OR icd_code = '3950') THEN 'ENDO_PVI'
            
            -- 6. CAROTID STENT (Target: Common K/L, Internal M/N)
            WHEN icd_version = 10 AND icd_code LIKE '037%' 
                 AND SUBSTRING(icd_code, 4, 1) IN ('K','L','M','N') THEN 'ENDO_CAS'
            WHEN icd_version = 9  AND (icd_code = '0061' OR icd_code = '0063') THEN 'ENDO_CAS'
        
            ELSE NULL
        END AS vascular_type
    FROM mimiciv_hosp.procedures_icd
),

exclusions AS (
    -- Exclude Cardiac, Thoracic, and Extra-Anatomic
    SELECT DISTINCT hadm_id
    FROM mimiciv_hosp.procedures_icd
    WHERE 
        (icd_version = 10 AND (icd_code LIKE '021%' OR icd_code LIKE '02R%')) OR
        (icd_version = 9  AND (icd_code LIKE '361%' OR icd_code LIKE '352%')) OR
        (icd_version = 10 AND icd_code LIKE '04V0%') OR 
        (icd_version = 10 AND icd_code LIKE '031%')
),

cohort_staging AS (
    SELECT 
        v.subject_id,
        v.hadm_id,
        MAX(CASE WHEN v.vascular_type LIKE 'OPEN%' THEN 1 ELSE 0 END) as has_open,
        MAX(CASE WHEN v.vascular_type LIKE 'ENDO%' THEN 1 ELSE 0 END) as has_endo
    FROM vascular_scans v
    WHERE v.vascular_type IS NOT NULL
      AND v.hadm_id NOT IN (SELECT hadm_id FROM exclusions)
    GROUP BY v.subject_id, v.hadm_id
)

SELECT 
    c.subject_id,
    c.hadm_id,
    a.admittime,
    a.dischtime,
    p.anchor_age,
    p.gender,
    CASE 
        WHEN c.has_open = 1 THEN 'OPEN'
        WHEN c.has_endo = 1 THEN 'ENDO'
        ELSE 'UNKNOWN'
    END as group_type
FROM cohort_staging c
JOIN mimiciv_hosp.admissions a ON c.hadm_id = a.hadm_id
JOIN mimiciv_hosp.patients p ON c.subject_id = p.subject_id
ORDER BY a.admittime;

**Query 2**

This was the query I used to populate the table for all vascular ICU patients

## N = 7676 ##

-- ------------------------------------------------------------------
-- SCRIPT 3: CREATE_STRICT_TABLE
-- ------------------------------------------------------------------

-- 1. Wipe the old version
DROP TABLE IF EXISTS vascular_cohort;

-- 2. Create the new Strict version
CREATE TABLE vascular_cohort AS
WITH vascular_scans AS (
    SELECT 
        subject_id,
        hadm_id,
        icd_code,
        icd_version,
        CASE
            -- GROUP A: OPEN
            WHEN icd_version = 10 AND (icd_code LIKE '04B0%' OR icd_code LIKE '04R0%') THEN 'OPEN_AAA'
            WHEN icd_version = 9  AND (icd_code = '3844') THEN 'OPEN_AAA'
            WHEN icd_version = 10 AND icd_code LIKE '041%' AND SUBSTRING(icd_code, 4, 1) NOT IN ('1','2','3','4','5','6','7','8') THEN 'OPEN_LE_BYPASS'
            WHEN icd_version = 9  AND (icd_code LIKE '3925' OR icd_code LIKE '3929') THEN 'OPEN_LE_BYPASS'
            WHEN icd_version = 10 AND icd_code LIKE '03C%' AND SUBSTRING(icd_code, 4, 1) IN ('K','L','M','N') THEN 'OPEN_CEA'
            WHEN icd_version = 9  AND icd_code = '3812' THEN 'OPEN_CEA'
            
            -- GROUP B: ENDO
            WHEN icd_version = 10 AND icd_code LIKE '04U0%' THEN 'ENDO_EVAR'
            WHEN icd_version = 9  AND icd_code = '3971' THEN 'ENDO_EVAR'
            WHEN icd_version = 10 AND (icd_code LIKE '047%' OR icd_code LIKE '04V%') AND SUBSTRING(icd_code, 4, 1) NOT IN ('1','2','3','4','5','6','7','8') THEN 'ENDO_PVI'
            WHEN icd_version = 9  AND (icd_code = '3990' OR icd_code = '3950') THEN 'ENDO_PVI'
            WHEN icd_version = 10 AND icd_code LIKE '037%' AND SUBSTRING(icd_code, 4, 1) IN ('K','L','M','N') THEN 'ENDO_CAS'
            WHEN icd_version = 9  AND (icd_code = '0061' OR icd_code = '0063') THEN 'ENDO_CAS'
            ELSE NULL
        END AS vascular_type
    FROM mimiciv_hosp.procedures_icd
),

exclusions AS (
    SELECT DISTINCT hadm_id FROM mimiciv_hosp.procedures_icd
    WHERE (icd_version = 10 AND (icd_code LIKE '021%' OR icd_code LIKE '02R%')) OR
          (icd_version = 9  AND (icd_code LIKE '361%' OR icd_code LIKE '352%')) OR
          (icd_version = 10 AND icd_code LIKE '04V0%') OR 
          (icd_version = 10 AND icd_code LIKE '031%')
),

cohort_staging AS (
    SELECT 
        v.subject_id,
        v.hadm_id,
        MAX(CASE WHEN v.vascular_type LIKE 'OPEN%' THEN 1 ELSE 0 END) as has_open,
        MAX(CASE WHEN v.vascular_type LIKE 'ENDO%' THEN 1 ELSE 0 END) as has_endo
    FROM vascular_scans v
    WHERE v.vascular_type IS NOT NULL
      AND v.hadm_id NOT IN (SELECT hadm_id FROM exclusions)
    GROUP BY v.subject_id, v.hadm_id
)

SELECT 
    c.subject_id,
    c.hadm_id,
    a.admittime,
    a.dischtime,
    p.anchor_age,
    p.gender,
    CASE 
        WHEN c.has_open = 1 THEN 'OPEN'
        WHEN c.has_endo = 1 THEN 'ENDO'
        ELSE 'UNKNOWN'
    END as group_type
FROM cohort_staging c
JOIN mimiciv_hosp.admissions a ON c.hadm_id = a.hadm_id
JOIN mimiciv_hosp.patients p ON c.subject_id = p.subject_id;

**Query 3**

This query pulled all the CR values for all of those patients (pre and post-op) --- ALL LAB VALUES

## 130k rows of data ##

-- ------------------------------------------------------------------
-- TITLE: EXTRACT_RAW_CREATININE
-- DESCRIPTION: Pulls all serum creatinine values (Pre and Post op)
-- ------------------------------------------------------------------

SELECT 
    vc.subject_id,
    vc.hadm_id,
    vc.group_type,
    le.charttime,
    le.valuenum as creatinine_value
FROM vascular_cohort vc
INNER JOIN mimiciv_hosp.labevents le
    ON vc.subject_id = le.subject_id -- Match on Patient, not just Admission (for Baseline lookback)
WHERE 
    le.itemid = 50912 -- Serum Creatinine
    AND le.valuenum IS NOT NULL
    -- Grab data from 1 year before admission (for Baseline) up to discharge
    AND le.charttime BETWEEN (vc.admittime - INTERVAL '365 days') AND vc.dischtime
ORDER BY vc.subject_id, le.charttime;

** Query 4 **
 Compared Day 0 cr to cr from a week before - (Gold standard)

-- ------------------------------------------------------------------
-- TITLE: DAY0_VS_RECENT_AUDIT
-- DESCRIPTION: Checks if Day 0 labs are hemodiluted compared to baseline
-- ------------------------------------------------------------------

WITH recent_labs AS (
    SELECT 
        vc.hadm_id,
        MAX(le.valuenum) as recent_cr 
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912
        AND le.valuenum IS NOT NULL
        -- The Gold Standard Window (Day -1 to -7)
        AND le.charttime::DATE >= (vc.admittime::DATE - 7)
        AND le.charttime::DATE < vc.admittime::DATE 
    GROUP BY vc.hadm_id
),
day0_labs AS (
    SELECT 
        vc.hadm_id,
        -- Take the FIRST lab of the day to try and catch them pre-fluids
        -- (Using MIN(charttime) logic would be better, but MAX val is safer for risk)
        MAX(le.valuenum) as day0_cr
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912
        AND le.valuenum IS NOT NULL
        -- Strictly Day 0
        AND le.charttime::DATE = vc.admittime::DATE
    GROUP BY vc.hadm_id
)

SELECT 
    COUNT(*) as n_overlap_patients,

    -- 1. DRIFT METRICS (Day 0 - Recent)
    -- Negative result = Day 0 is lower (Dilution)
    ROUND(AVG(day0.day0_cr - recent.recent_cr)::NUMERIC, 3) as mean_drift_mg_dl,
    ROUND(AVG(ABS(day0.day0_cr - recent.recent_cr))::NUMERIC, 3) as mean_absolute_error,

    -- 2. THE "HIDDEN CKD" CHECK (The most dangerous metric)
    -- Scenario: Recent was High (>=1.3), but Day 0 looks Normal (<1.3)
    SUM(CASE 
        WHEN recent.recent_cr >= 1.3 AND day0.day0_cr < 1.3 THEN 1 ELSE 0 
    END) as count_diluted_normal,
    
    ROUND(
        (SUM(CASE WHEN recent.recent_cr >= 1.3 AND day0.day0_cr < 1.3 THEN 1 ELSE 0 END) * 100.0 / COUNT(*))::NUMERIC, 
        1
    ) as pct_diluted_normal

FROM recent_labs recent
INNER JOIN day0_labs day0 ON recent.hadm_id = day0.hadm_id;

** Query 5 **

Query that startified the cohort in Recent, Day0, and Remote (still 365 days)

-- ------------------------------------------------------------------
-- TITLE: FINAL_BASELINE_CONSTRUCTION
-- DESCRIPTION: Coalesces pre-op data based on MAE validation (Recent > Day0 > Remote)
-- ------------------------------------------------------------------

WITH ranked_labs AS (
    SELECT 
        vc.hadm_id,
        le.valuenum as lab_value,
        le.charttime,
        vc.admittime,
        -- Calculate the "Days Prior" for reference
        (le.charttime::DATE - vc.admittime::DATE) as days_diff,
        
        -- ASSIGN PRIORITY BASED ON AUDIT RESULTS
        CASE 
            -- Priority 1: Recent (Gold Standard)
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 7) 
             AND le.charttime::DATE < vc.admittime::DATE 
            THEN 1
            
            -- Priority 2: Day of Surgery (Silver Standard - MAE 0.30)
            WHEN le.charttime::DATE = vc.admittime::DATE 
            THEN 2
            
            -- Priority 3: Remote (Bronze Standard - MAE 0.63)
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 365) 
             AND le.charttime::DATE <= (vc.admittime::DATE - 8)
            THEN 3
            
            ELSE 4 -- Out of window
        END as priority_rank
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912 -- Creatinine
        AND le.valuenum IS NOT NULL
        AND le.charttime >= (vc.admittime - INTERVAL '365 days')
        AND le.charttime <= vc.admittime
)

SELECT 
    vc.hadm_id,
    
    -- 1. The Chosen Value
    MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 1) as cr_recent,
    MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 2) as cr_day0,
    MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 3) as cr_remote,
    
    -- 2. The Final Coalesced Column (This is your model input)
    COALESCE(
        MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 1), -- Try Recent
        MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 2), -- Fallback to Day 0
        MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 3)  -- Last resort Remote
    ) as baseline_creatinine,
    
    -- 3. The Source Flag (For your "Limitations" section)
    CASE 
        WHEN MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 1) IS NOT NULL THEN 'Recent'
        WHEN MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 2) IS NOT NULL THEN 'Day0'
        WHEN MAX(rl.lab_value) FILTER (WHERE rl.priority_rank = 3) IS NOT NULL THEN 'Remote'
        ELSE 'Missing'
    END as data_source

FROM vascular_cohort vc
LEFT JOIN ranked_labs rl ON vc.hadm_id = rl.hadm_id
GROUP BY vc.hadm_id;

** Query 6 **

Calculated patients with AKI (Pre 90 day limitation)

Stage 3 AKI - 615 (9%) (9% post 90 limitation)
Stage 2 AKI - 102 (1.5%) (1.4% post limitation)
Stage 1 AKI - 763 (11.3%) (10.9% post limitation)

-- ------------------------------------------------------------------
-- TITLE: MASTER_VASCULAR_AKI_LOGIC
-- DESCRIPTION: 1. Builds Baseline (Recent > Day0 > Remote)
--              2. Calculates AKI (KDIGO Stages)
-- ------------------------------------------------------------------

WITH ranked_baselines AS (
    -- STEP 1: FIND ALL POTENTIAL BASELINES
    SELECT 
        vc.hadm_id,
        vc.subject_id,
        vc.admittime,
        le.valuenum as lab_value,
        le.charttime,
        
        -- Assign Priority
        CASE 
            -- Priority 1: Recent (Gold) - Day -1 to -7
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 7) 
             AND le.charttime::DATE < vc.admittime::DATE 
            THEN 1
            
            -- Priority 2: Day of Surgery (Silver) - Day 0
            WHEN le.charttime::DATE = vc.admittime::DATE 
            THEN 2
            
            -- Priority 3: Remote (Bronze) - Day -8 to -365
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 365) 
             AND le.charttime::DATE <= (vc.admittime::DATE - 8)
            THEN 3
            
            ELSE 4 
        END as priority_rank
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912 -- Creatinine
        AND le.valuenum IS NOT NULL
        AND le.charttime >= (vc.admittime - INTERVAL '365 days')
        AND le.charttime <= vc.admittime
),
best_baseline AS (
    -- STEP 2: SELECT THE SINGLE BEST BASELINE PER PATIENT
    SELECT 
        hadm_id,
        subject_id,
        admittime,
        -- Pick the lab with the lowest Rank (1 is best)
        -- If ties (e.g. 2 labs on same day), pick the latest one (charttime DESC)
        (ARRAY_AGG(lab_value ORDER BY priority_rank ASC, charttime DESC))[1] as baseline_creatinine,
        
        CASE 
            WHEN min(priority_rank) = 1 THEN 'Recent'
            WHEN min(priority_rank) = 2 THEN 'Day0'
            WHEN min(priority_rank) = 3 THEN 'Remote'
            ELSE 'Missing'
        END as data_source
    FROM ranked_baselines
    GROUP BY hadm_id, subject_id, admittime
),
post_op_max AS (
    -- STEP 3: FIND PEAK POST-OP CREATININE
    SELECT 
        bb.hadm_id,
        bb.baseline_creatinine,
        bb.data_source,
        MAX(le.valuenum) as max_postop_cr
    FROM best_baseline bb
    LEFT JOIN mimiciv_hosp.labevents le 
        ON bb.subject_id = le.subject_id 
        AND le.itemid = 50912 
        AND le.valuenum IS NOT NULL
        
        -- DYNAMIC START TIME:
        -- If Baseline was 'Day0', start looking on Day 1 (Post-op Day 1)
        -- Otherwise start looking on Day 0 (Post-op Day 0)
        AND le.charttime::DATE >= CASE 
            WHEN bb.data_source = 'Day0' THEN (bb.admittime::DATE + 1)
            ELSE bb.admittime::DATE
        END
        
        -- END TIME: 7 Days after surgery
        AND le.charttime::DATE <= (bb.admittime::DATE + 7)
        
    WHERE bb.baseline_creatinine IS NOT NULL -- Exclude the missing people
    GROUP BY bb.hadm_id, bb.baseline_creatinine, bb.data_source
)

-- STEP 4: CALCULATE FINAL STAGES
SELECT 
    CASE 
        -- Stage 3
        WHEN (max_postop_cr / baseline_creatinine) >= 3.0 
          OR (max_postop_cr >= 4.0 AND (max_postop_cr - baseline_creatinine) >= 0.3) 
        THEN 3
        
        -- Stage 2
        WHEN (max_postop_cr / baseline_creatinine) >= 2.0 
        THEN 2
        
        -- Stage 1
        WHEN (max_postop_cr / baseline_creatinine) >= 1.5 
          OR (max_postop_cr - baseline_creatinine) >= 0.3 
        THEN 1
        
        ELSE 0 
    END as aki_stage,
    
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as percentage

FROM post_op_max
GROUP BY 1
ORDER BY 1 DESC;

**Query 7**

####OFFICIAL COHORT CREATION####

After droping patients with no cr baseline or cr more than 90 prior

##N = 6281##

DATA source colum:
            WHEN min(priority_rank) = 1 THEN 'Recent'
            WHEN min(priority_rank) = 2 THEN 'Day0'
            WHEN min(priority_rank) = 3 THEN 'Remote (<90d)'
            ELSE 'Missing'

-- ------------------------------------------------------------------
-- TITLE: STRICT_BASELINE_COHORT (90-DAY LIMIT)
-- DESCRIPTION: Re-builds the cohort. Drops Missing. Drops >90 Day Remote.
-- ------------------------------------------------------------------

DROP TABLE IF EXISTS vascular_cohort_strict;

CREATE TABLE vascular_cohort_strict AS
WITH ranked_baselines AS (
    SELECT 
        vc.hadm_id,
        vc.subject_id,
        vc.admittime,
        le.valuenum as lab_value,
        le.charttime,
        
        -- PRIORITY LOGIC (Updated)
        CASE 
            -- Priority 1: Recent (-1 to -7 days)
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 7) 
             AND le.charttime::DATE < vc.admittime::DATE 
            THEN 1
            
            -- Priority 2: Day of Surgery (Day 0)
            WHEN le.charttime::DATE = vc.admittime::DATE 
            THEN 2
            
            -- Priority 3: STRICT REMOTE (-8 to -90 days ONLY)
            -- We changed the window from 365 to 90
            WHEN le.charttime::DATE >= (vc.admittime::DATE - 90) 
             AND le.charttime::DATE <= (vc.admittime::DATE - 8)
            THEN 3
            
            ELSE 4 -- Anything older than 90 days is now 'Priority 4' (Ignored)
        END as priority_rank
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912 -- Creatinine
        AND le.valuenum IS NOT NULL
        -- Hard Filter: Only look back 90 days max
        AND le.charttime >= (vc.admittime - INTERVAL '90 days')
        AND le.charttime <= vc.admittime
),
best_baseline AS (
    SELECT 
        hadm_id,
        subject_id,
        admittime,
        -- Select the BEST single value
        (ARRAY_AGG(lab_value ORDER BY priority_rank ASC, charttime DESC))[1] as baseline_creatinine,
        
        -- Label the source
        CASE 
            WHEN min(priority_rank) = 1 THEN 'Recent'
            WHEN min(priority_rank) = 2 THEN 'Day0'
            WHEN min(priority_rank) = 3 THEN 'Remote (<90d)'
            ELSE 'Missing'
        END as data_source
    FROM ranked_baselines
    GROUP BY hadm_id, subject_id, admittime
)

-- FINAL FILTER: Drop the Missing
SELECT * FROM best_baseline
WHERE baseline_creatinine IS NOT NULL;

** Query 8 **

compares the 8-90 cr to the 1-7 day group

-- ------------------------------------------------------------------
-- TITLE: STRICT_REMOTE_VS_RECENT_AUDIT
-- DESCRIPTION: Compares 8-90 day labs against the -1 to -7 day Gold Standard
-- ------------------------------------------------------------------

WITH recent_labs AS (
    SELECT 
        vc.hadm_id,
        MAX(le.valuenum) as recent_cr 
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912
        AND le.valuenum IS NOT NULL
        -- Gold Standard Window
        AND le.charttime::DATE >= (vc.admittime::DATE - 7)
        AND le.charttime::DATE < vc.admittime::DATE 
    GROUP BY vc.hadm_id
),
strict_remote_labs AS (
    SELECT 
        vc.hadm_id,
        -- Take the LATEST lab in the window (closest to surgery)
        (ARRAY_AGG(le.valuenum ORDER BY le.charttime DESC))[1] as remote_cr
    FROM vascular_cohort vc
    JOIN mimiciv_hosp.labevents le 
        ON vc.subject_id = le.subject_id 
        AND le.itemid = 50912
        AND le.valuenum IS NOT NULL
        -- The "Strict" Window (8 to 90 days)
        AND le.charttime::DATE >= (vc.admittime::DATE - 90)
        AND le.charttime::DATE <= (vc.admittime::DATE - 8)
    GROUP BY vc.hadm_id
)

SELECT 
    COUNT(*) as n_overlap_patients,

    -- 1. ERROR METRICS
    -- Mean Absolute Error: How far off is the "Old" lab on average?
    ROUND(AVG(ABS(rm.remote_cr - rc.recent_cr))::NUMERIC, 3) as mean_absolute_error,
    
    -- Mean Drift: Direction of error (Negative = Old lab was lower/better)
    ROUND(AVG(rm.remote_cr - rc.recent_cr)::NUMERIC, 3) as mean_drift,

    -- 2. CLINICAL IMPACT
    -- How many patients had a clinically significant shift (>0.3)?
    SUM(CASE WHEN ABS(rm.remote_cr - rc.recent_cr) >= 0.3 THEN 1 ELSE 0 END) as count_significant_change,
    
    ROUND(
        (SUM(CASE WHEN ABS(rm.remote_cr - rc.recent_cr) >= 0.3 THEN 1 ELSE 0 END) * 100.0 / COUNT(*))::NUMERIC, 
        1
    ) as pct_significant_change

FROM recent_labs rc
INNER JOIN strict_remote_labs rm ON rc.hadm_id = rm.hadm_id;